In [1]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=false
α= α3b(fermion,0.0,0.0,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.0 T=0.0 parity=1 Number of channels: 1
scaling factor for x: 0.3


Main.mesh.meshset(12, 12, 12, [-0.9815606342467192, -0.9041172563704748, -0.7699026741943047, -0.5873179542866175, -0.3678314989981802, -0.1252334085114689, 0.1252334085114689, 0.3678314989981802, 0.5873179542866175, 0.7699026741943047, 0.9041172563704748, 0.9815606342467192], [0.04717533638651175, 0.10693932599531826, 0.16007832854334625, 0.20316742672306587, 0.23349253653835492, 0.24914704581340288, 0.24914704581340288, 0.23349253653835492, 0.20316742672306587, 0.16007832854334625, 0.10693932599531826, 0.04717533638651175], [0.0848575044719742, 0.2856978124093846, 0.6049476415733046, 1.0477062209334076, 1.6216473060047178, 2.337844182123722, 3.2122166066967295, 4.268145709136995, 5.5415989902690015, 7.092535125720268, 9.036017587831902, 11.666785312828601], [0.14277897009800458, 0.259383321803658, 0.37995967777078926, 0.5068231073270444, 0.6428816610948332, 0.7921245262743847, 0.960438937417142, 1.1572465340299454, 1.3992307171257872, 1.7204236864089044, 2.2079451522012565, 3.2157606

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
Gαα=computeGcoefficient(α, grid)

In [3]:
using Revise
includet("twobody.jl")
using .twobodybound

In [4]:
potname="MT"
bound_states=bound2b(grid, potname)

For J=1.0 parity=+ Number of channels: 2
Number of bound states: 1
Bound state energies: [-2.220428332024142]


1-element Vector{Any}:
 (-2.220428332024142, [-0.0033236481027346695, -0.03648135266525975, -0.152039580930092, -0.32120450972445946, -0.4311612801842848, -0.45278501208225486, -0.41885803539313576, -0.36277768667592314, -0.2954622232673801, -0.22833076783345466  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [7]:
using Revise
includet("matrices.jl")
using .matrices


In [8]:
Rxy_matrix(α, grid)


144×144 Matrix{ComplexF64}:
      13.4046+0.0im      -0.28484+0.0im  …  -4.00884e-7+0.0im
    -0.388215+0.0im      0.158143+0.0im      1.32599e-7+0.0im
      0.06072+0.0im      0.014069+0.0im      4.06535e-7+0.0im
    0.0127869+0.0im    -0.0149884+0.0im       2.9187e-7+0.0im
   0.00083975+0.0im  -0.000858678+0.0im      2.18607e-8+0.0im
  -0.00100088+0.0im   0.000645621+0.0im  …  -2.01881e-7+0.0im
  -0.00068003+0.0im   0.000427737+0.0im     -2.74591e-7+0.0im
 -0.000241174+0.0im   0.000147494+0.0im     -1.96499e-7+0.0im
  -2.58379e-5+0.0im    1.59816e-5+0.0im      -2.9225e-8+0.0im
   3.21404e-5+0.0im   -1.83673e-5+0.0im      1.70686e-7+0.0im
             ⋮                           ⋱  
   -0.0458766+0.0im     0.0256066+0.0im  …  -0.00104481+0.0im
   -0.0468156+0.0im     0.0261028+0.0im     -0.00103494+0.0im
    0.0347493+0.0im    -0.0194446+0.0im     0.000957861+0.0im
    0.0569587+0.0im      -0.03191+0.0im      0.00221529+0.0im
     0.083299+0.0im    -0.0465384+0.0im      0.00349868+0.0

In [15]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [16]:
ThreeBody_Bound(α, grid)

12-element Vector{Any}:
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
 (-43.89847796640892, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0

In [9]:
T_matrix(α,grid) 

144×144 Matrix{Float64}:
 14083.7      -1667.48      235.522    …    0.0         0.0       0.0
 -1667.48      9595.01     -591.035         0.0         0.0       0.0
   235.522     -591.035    8739.59          0.0         0.0       0.0
   -40.4788      83.1169   -296.602         0.0         0.0       0.0
    -9.28087    -10.7471     41.4457        0.0         0.0       0.0
    24.846       -8.00025    -3.83259  …    0.0         0.0       0.0
   -29.5417      13.7864     -6.0605        0.0         0.0       0.0
    30.1921     -15.3741      9.09489       0.0         0.0       0.0
   -29.2067      15.3743     -9.865         0.0         0.0       0.0
    27.5315     -14.7187      9.76021      31.0602      0.0       0.0
     ⋮                                 ⋱                         
     0.0          0.0         0.0      …   -6.96166     6.82188  -6.40611
     0.0          0.0         0.0           4.9691     -5.19224   5.01381
     0.0          0.0         0.0          -3.17681     3.921

In [10]:
V_matrix(α, grid, "MT")

144×144 Kronecker.KroneckerProduct{Float64, Matrix{Float64}, Matrix{Float64}}:
 7711.5     0.0     0.0     0.0     0.0     0.0  …   0.0          0.0
    0.0  7711.5     0.0     0.0     0.0     0.0      0.0          0.0
    0.0     0.0  7711.5     0.0     0.0     0.0      0.0          0.0
    0.0     0.0     0.0  7711.5     0.0     0.0      0.0          0.0
    0.0     0.0     0.0     0.0  7711.5     0.0      0.0          0.0
    0.0     0.0     0.0     0.0     0.0  7711.5  …   0.0          0.0
    0.0     0.0     0.0     0.0     0.0     0.0      0.0          0.0
    0.0     0.0     0.0     0.0     0.0     0.0      0.0          0.0
    0.0     0.0     0.0     0.0     0.0     0.0      0.0          0.0
    0.0     0.0     0.0     0.0     0.0     0.0      0.0          0.0
    ⋮                                       ⋮    ⋱               
    0.0     0.0     0.0     0.0     0.0     0.0  …  -0.0         -0.0
    0.0     0.0     0.0     0.0     0.0     0.0     -0.0         -0.0
    0.0     0.0